# Activity 3.2 - Transfer Learning

Submitted By: Labbao, Benedick D.<br>
Performed On: 03/05/2024<br>
Submitted On:

Submitted To: Engr. Roman M. Richard

---

#### Objective(s):

This activity aims to introduce how to apply transfer learning

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [54]:
now = datetime.datetime.now
batch_size = 16
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [55]:

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps.
* Use the model, training set, test set and number of classes as function parameters


In [56]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [48]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Create two datasets
* one with digits below 5
* one with 5 and above

In [49]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [ ]:


feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [ ]:


classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [ ]:

model = Sequential(feature_layers + classification_layers)

Check the model summary

In [ ]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

 Train the  model on the digits 5,6,7,8,9

In [ ]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 46s 194ms/step - loss: 1.6238 - accuracy: 0.1933 - val_loss: 1.5982 - val_accuracy: 0.2177
Epoch 2/5
230/230 [==============================] - 46s 200ms/step - loss: 1.5912 - accuracy: 0.2467 - val_loss: 1.5630 - val_accuracy: 0.2892
Epoch 3/5
230/230 [==============================] - 45s 197ms/step - loss: 1.5577 - accuracy: 0.3190 - val_loss: 1.5253 - val_accuracy: 0.4551
Epoch 4/5
230/230 [==============================] - 45s 197ms/step - loss: 1.5230 - accuracy: 0.3869 - val_loss: 1.4844 - val_accuracy: 0.6258
Epoch 5/5
230/230 [==============================] - 47s 205ms/step - loss: 1.4841 - accuracy: 0.4616 - val_loss: 1.4389 - val_accuracy: 0.7029
Training time: 0:04:23.184993
Test score: 1.4389315843582153
Test accuracy: 0.7029417753219604


Freeze only the feature layers

In [ ]:

for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

Train again the model using the 0 to 4 digits

In [ ]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 21s 85ms/step - loss: 1.5665 - accuracy: 0.3416 - val_loss: 1.5192 - val_accuracy: 0.6153
Epoch 2/5
240/240 [==============================] - 18s 73ms/step - loss: 1.5067 - accuracy: 0.4443 - val_loss: 1.4564 - val_accuracy: 0.7001
Epoch 3/5
240/240 [==============================] - 18s 73ms/step - loss: 1.4506 - accuracy: 0.5288 - val_loss: 1.3967 - val_accuracy: 0.7435
Epoch 4/5
240/240 [==============================] - 17s 70ms/step - loss: 1.3965 - accuracy: 0.5924 - val_loss: 1.3387 - val_accuracy: 0.7980
Epoch 5/5
240/240 [==============================] - 16s 69ms/step - loss: 1.3425 - accuracy: 0.6536 - val_loss: 1.2820 - val_accuracy: 0.8387
Training time: 0:02:23.070042
Test score: 1.2820041179656982
Test accuracy: 0.8386845588684082


#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [57]:

# Create feature and
# classification layers
# to use

feature_layers_2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers_2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [58]:
# Create a Sequential model
# using the layers created
# from before

model_2 = Sequential(feature_layers_2 + classification_layers_2)

In [59]:
# check the summary of
# model before freezing
# feature layers

model_2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 activation_24 (Activation)  (None, 26, 26, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 24, 24, 32)        9248      
                                                                 
 activation_25 (Activation)  (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 12, 12, 32)        0         
                                                      

In [60]:
# train the model using
# data from less than 5

train_model(model_2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
1913/1913 [==============================] - 80s 42ms/step - loss: 1.5103 - accuracy: 0.4466 - val_loss: 1.3964 - val_accuracy: 0.8033
Epoch 2/5
1913/1913 [==============================] - 87s 46ms/step - loss: 1.3084 - accuracy: 0.6837 - val_loss: 1.1366 - val_accuracy: 0.8770
Epoch 3/5
1913/1913 [==============================] - 86s 45ms/step - loss: 1.0401 - accuracy: 0.7923 - val_loss: 0.8149 - val_accuracy: 0.9062
Epoch 4/5
1913/1913 [==============================] - 84s 44ms/step - loss: 0.7622 - accuracy: 0.8415 - val_loss: 0.5393 - val_accuracy: 0.9208
Epoch 5/5
1913/1913 [==============================] - 75s 39ms/step - loss: 0.5612 - accuracy: 0.8673 - val_loss: 0.3729 - val_accuracy: 0.9331
Training time: 0:06:52.890358
Test score: 0.37294065952301025
Test accuracy: 0.933060884475708


In [61]:
# freeze feature layers

for l in feature_layers_2:
        l.trainable = False

In [62]:
# check the summary of
# model after freezing
# feature layers

model_2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 activation_24 (Activation)  (None, 26, 26, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 24, 24, 32)        9248      
                                                                 
 activation_25 (Activation)  (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 12, 12, 32)        0         
                                                      

In [63]:
# train the model using
# data from greater than
# or equal to 5 while feature
# layers are frozen to
# fine-tune the model

train_model(model_2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
1838/1838 [==============================] - 41s 22ms/step - loss: 1.7069 - accuracy: 0.3415 - val_loss: 1.3835 - val_accuracy: 0.4378
Epoch 2/5
1838/1838 [==============================] - 35s 19ms/step - loss: 1.3179 - accuracy: 0.4668 - val_loss: 1.0687 - val_accuracy: 0.6268
Epoch 3/5
1838/1838 [==============================] - 38s 20ms/step - loss: 1.0754 - accuracy: 0.6068 - val_loss: 0.8672 - val_accuracy: 0.7486
Epoch 4/5
1838/1838 [==============================] - 37s 20ms/step - loss: 0.9129 - accuracy: 0.6972 - val_loss: 0.7322 - val_accuracy: 0.8062
Epoch 5/5
1838/1838 [==============================] - 38s 21ms/step - loss: 0.7946 - accuracy: 0.7544 - val_loss: 0.6386 - val_accuracy: 0.8360
Training time: 0:03:23.286299
Test score: 0.6385741233825684
Test accuracy: 0.8360419869422913


# Conclusion

In this activity, we demonstrated the possible of taking only the features of the data and using it in other classification, in the example that we used, we trained a model to classify numbers 0-4 and then fine-tuned it to be used for classification of numbers 5-9. The result showed increase in test accuracy, which indicates that Transfer Learning technique is working. In conclusion, using Transfer Learning can reduce resources used for training by using pretrained model.